<a href="https://colab.research.google.com/github/LCaravaggio/AnalisisCuantitativoAvanzado/blob/main/SeriesdeTiempo/EMAE_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json

import requests
from requests.auth import HTTPBasicAuth

import pandas as pd

from matplotlib import pyplot as plt
import seaborn as sns

In [2]:
from google.colab import userdata

# Definir user key para la API
user_key = userdata.get('ALPHACAST')

In [3]:
# EMAE
emae = requests.get('https://api.alphacast.io/datasets/5331/data?apiKey='+user_key+'&&$format=csv', auth=HTTPBasicAuth(user_key, ''))
open('emae.csv','wb').write(emae.content)
emae = pd.read_csv('emae.csv',delimiter = ',')

In [4]:
emae.set_index('Date', inplace = True)
emae = emae.loc['2016-02-01':]
y=emae['Emae - sa_orig - current_prices_mom']

In [5]:
y

,Emae - sa_orig - current_prices_mom
Date,
2016-02-01,-0.627192
2016-03-01,-0.670262
2016-04-01,-0.771878
2016-05-01,-0.290096
2016-06-01,-0.225846
...,...
2025-03-01,-1.668469
2025-04-01,1.215458
2025-05-01,-0.244393


In [6]:
# IPI
ipi = requests.get('https://api.alphacast.io/datasets/5679/data?apiKey='+user_key+'&&$format=csv', auth=HTTPBasicAuth(user_key, ''))
open('ipi.csv','wb').write(ipi.content)
ipi = pd.read_csv('ipi.csv',delimiter = ',')


ipi['Date'] = pd.to_datetime(ipi['Date'])
ipi.set_index('Date', inplace = True)

In [7]:
ipi.head()

,Country,Division,Seasonal Adjustment,Subclass,Value,Value - current_prices_3m_yoy,Value - current_prices_yoy,Value - current_prices_mom
Date,,,,,,,,
2016-01-01,Argentina,Alimentos y bebidas,Not adjusted,"Azúcar, productos de confitería y chocolate",50.680600,NaN,NaN,NaN
2016-01-01,Argentina,Alimentos y bebidas,Not adjusted,Carne aviar,212.009238,NaN,NaN,NaN
2016-01-01,Argentina,Alimentos y bebidas,Not adjusted,Carne vacuna,83.169668,NaN,NaN,NaN
2016-01-01,Argentina,Alimentos y bebidas,Not adjusted,Fiambres y embutidos,143.350871,NaN,NaN,NaN
2016-01-01,Argentina,Alimentos y bebidas,Not adjusted,"Galletitas, productos de panadería y pastas",95.801964,NaN,NaN,NaN


In [8]:
ipi[(ipi['Seasonal Adjustment']== "Seasonally Adjusted")].head()

,Country,Division,Seasonal Adjustment,Subclass,Value,Value - current_prices_3m_yoy,Value - current_prices_yoy,Value - current_prices_mom
Date,,,,,,,,
2016-01-01,Argentina,IPI Manufacturero,Seasonally Adjusted,none,134.582607,NaN,NaN,NaN
2016-02-01,Argentina,IPI Manufacturero,Seasonally Adjusted,none,133.593795,NaN,NaN,-0.734725
2016-03-01,Argentina,IPI Manufacturero,Seasonally Adjusted,none,130.799171,NaN,NaN,-2.091882
2016-04-01,Argentina,IPI Manufacturero,Seasonally Adjusted,none,130.585389,NaN,NaN,-0.163443
2016-05-01,Argentina,IPI Manufacturero,Seasonally Adjusted,none,128.706060,NaN,NaN,-1.439157


In [9]:
ipi = ipi[(ipi['Division'] == "IPI Manufacturero") & (ipi['Seasonal Adjustment'] == "Seasonally Adjusted")]
ipi = ipi['Value - current_prices_mom']
ipi.columns = ['Value - sa_MoM']
ipi = ipi.loc['2016-02-01':]

In [10]:
# ISAC
isac = requests.get('https://api.alphacast.io/datasets/5565/data?apiKey='+user_key+'&&$format=csv', auth=HTTPBasicAuth(user_key, ''))
open('isac.csv','wb').write(isac.content)
isac = pd.read_csv('isac.csv',delimiter = ',')

In [11]:
isac['Date'] = pd.to_datetime(isac['Date'])
isac.set_index('Date', inplace = True)
isac = isac['Nivel general - sa_orig - current_prices_mom']
isac.columns = ['Nivel general - sa_MoM']
isac = isac.loc['2016-02-01':]

In [12]:
isac=pd.Series(isac, name='ISAC')
ipi=pd.Series(ipi, name='IPI')

In [13]:
X = pd.concat([ipi, isac], axis=1)

In [14]:
X

,IPI,ISAC
Date,,
2016-02-01,-0.734725,-2.073035
2016-03-01,-2.091882,-8.455584
2016-04-01,-0.163443,-4.559165
2016-05-01,-1.439157,4.765745
2016-06-01,-3.140229,-4.156349
...,...,...
2025-03-01,-4.063489,-3.636319
2025-04-01,2.474633,5.052270
2025-05-01,2.545213,-1.821139


In [15]:
y.shape

(114,)

In [16]:
X.shape

(114, 2)

In [17]:
from sklearn import linear_model
lr = linear_model.LinearRegression()
lr.fit(X[:-1], y[:-1]) #Si ya salió IPI e ISAC, pero todavía no el EMAE entonces hay que fitear lr.fit(X[:-1], y) si ya salió todo, lr.fit(X[:-1], y[:-1])
Yest = lr.predict(X[:-1])

In [18]:
Yest

array([-3.37523554e-01, -1.11646175e+00, -2.05349780e-01, -3.57744498e-01,
       -1.36896201e+00,  8.19594663e-01, -6.18563116e-01,  1.37547752e+00,
       -4.47234668e-01,  1.17566557e+00,  1.17789334e-01, -2.91714728e-01,
       -1.38614804e+00,  1.53479406e+00,  6.51784627e-02,  7.18906516e-01,
        6.99578079e-01,  6.53093757e-01, -3.16622785e-01,  9.15227896e-01,
       -4.49846020e-01,  1.17887060e+00, -3.85913794e-01, -1.45318433e+00,
       -2.52057557e-01,  9.91122132e-01, -2.78633957e-01, -7.26225020e-01,
       -1.74862480e+00, -6.04723238e-01,  3.39095936e-01, -4.25344551e-01,
       -9.04389749e-01, -1.14267667e+00, -3.47080069e-01,  7.85974207e-01,
        1.10022173e+00, -8.67640957e-01,  1.49140817e-01,  8.90274483e-01,
       -6.36287074e-01, -2.91374614e-01,  1.35109456e-01, -1.42823187e+00,
        1.29018935e+00, -1.24568402e+00,  1.24700816e-03,  4.98219497e-01,
        5.82471197e-01, -9.16909205e+00, -9.03575148e+00,  1.29601943e+01,
        6.96209055e+00,  

In [19]:
print('DATOS DEL MODELO REGRESIÓN LINEAL SIMPLE')
print()
print('Valor de la pendiente o coeficiente "a":')
print(lr.coef_)
print('Valor de la intersección o coeficiente "b":')
print(lr.intercept_)
print()
print('La ecuación del modelo es igual a:')
print('y = ', lr.coef_, 'X + ', lr.intercept_)
print()
print('Precisión del modelo:')
print(lr.score(X[:-1], y[:-1]))

DATOS DEL MODELO REGRESIÓN LINEAL SIMPLE

Valor de la pendiente o coeficiente "a":
[0.3960153  0.03783494]
Valor de la intersección o coeficiente "b":
0.03187193350045088

La ecuación del modelo es igual a:
y =  [0.3960153  0.03783494] X +  0.03187193350045088

Precisión del modelo:
0.7631324715337406


In [20]:
lr.coef_[0]

np.float64(0.3960153023988825)

In [21]:
X[-1:]

,IPI,ISAC
Date,,
2025-07-01,-2.29136,-1.817049


In [22]:
emae_predict=X[-1:]['IPI']*lr.coef_[0]+X[-1:]['ISAC']*lr.coef_[1]+lr.intercept_

In [23]:
emae_predict

,0
Date,
2025-07-01,-0.94429
